In [1]:
nothing here

[1, 2, 3]


import inspect
import ast

ast_string = ast.dump(ast.parse("[1,2,3]"))


def to_ast(d):
    if isinstance(d, ast.Call):
        return getattr(ast, d.func.id)(
            *map(to_ast, d.args),
            **{i.arg: to_ast(i.value) for i in d.keywords},
            **{
                "lineno": None,
                "col_offset": None,
                "end_lineno": None,
                "end_col_offset": None,
            },
        )
    if isinstance(d, ast.List):
        return list(map(to_ast, d.elts))
    return d.value


def ast_to_code(ast_string):
    return ast.unparse(to_ast(ast.parse(ast_string).body[0].value))


import ast, re


def ast_to_code(ast_string):
    return ast.unparse(
        eval(re.sub("\w+(?=\()", lambda x: f"ast.{x.group()}", ast_string))
    )


ast_string = ast.dump(ast.parse("[1,2,3]"))
print(ast_to_code(ast_string))

In [135]:
a = (i for i in range(10))
print("test")

test


In [136]:
def gen():
    a = 1
    yield a
    a = 4
    yield a + 1


print("test")

test


In [137]:
g = gen()  # Create the generator
next(g)

1

In [139]:
t = executing.Source.executing(a.gi_frame).source.asttokens()
t.tokens

[Token(type=1, string='a', start=(1, 0), end=(1, 1), line='a = (i for i in range(10))\n', index=0, startpos=0, endpos=1),
 Token(type=54, string='=', start=(1, 2), end=(1, 3), line='a = (i for i in range(10))\n', index=1, startpos=2, endpos=3),
 Token(type=54, string='(', start=(1, 4), end=(1, 5), line='a = (i for i in range(10))\n', index=2, startpos=4, endpos=5),
 Token(type=1, string='i', start=(1, 5), end=(1, 6), line='a = (i for i in range(10))\n', index=3, startpos=5, endpos=6),
 Token(type=1, string='for', start=(1, 7), end=(1, 10), line='a = (i for i in range(10))\n', index=4, startpos=7, endpos=10),
 Token(type=1, string='i', start=(1, 11), end=(1, 12), line='a = (i for i in range(10))\n', index=5, startpos=11, endpos=12),
 Token(type=1, string='in', start=(1, 13), end=(1, 15), line='a = (i for i in range(10))\n', index=6, startpos=13, endpos=15),
 Token(type=1, string='range', start=(1, 16), end=(1, 21), line='a = (i for i in range(10))\n', index=7, startpos=16, endpos=21),
 

In [378]:
def my_function():
    print("Hello, World!")


# Get the source code
source_code = inspect.getsource(gen)
print(source_code)

def gen():
    a = 1
    yield a
    a = 4
    yield a + 1



In [2]:
import inspect


def my_generator():
    for i in range(5):
        yield i


# Create a generator instance
a = my_generator()

# Get the source code of the generator function
source_code = inspect.getsource(a.gi_code)
print(source_code)

def my_generator():
    for i in range(5):
        yield i



In [3]:
current_line = a.gi_frame.f_lineno
current_line

4

In [4]:
# Access the generator's frame
frame = inspect.getgeneratorstate(g)
print(frame)

NameError: name 'g' is not defined

In [88]:
frame = g.gi_frame
print(frame.f_locals)

{'a': 1}


In [89]:
executing.Source.executing(g.gi_frame).source.lines

['def gen():', '    a = 1', '    yield a', '    a = 4', '    yield a + 1']

In [125]:
t = executing.Source.executing(g.gi_frame).frame

<frame at 0x7fffb9e07f60, file '/tmp/ipykernel_526/25615.py', line 3, code gen>

In [215]:
def my_generator():
    a = 1
    yield a
    b = a + 1
    for i in range(5):
        yield i


test = my_generator()
next(test)
next(test)

1

In [218]:
next(test)

0

In [220]:
test.gi_frame.f_lineno = 3

ValueError: f_lineno can only be set by a trace function

In [ ]:
import sys


def trace_calls(frame, event, arg):
    if event == "call":
        print(f"Function called: {frame.f_code.co_name} on line {frame.f_lineno}")
    elif event == "line":
        print(f"Line {frame.f_lineno}")
    return trace_calls


sys.settrace(trace_calls)

In [182]:
inspect.getsource(test.gi_code)

'def my_generator():\n    a = 1\n    yield a\n    b = a+1\n    for i in range(5):\n        yield i\n'

In [213]:
inspect.getsource

<function inspect.getsource(object)>

In [183]:
test.gi_frame.f_lineno

6

In [184]:
next(test)

1

In [212]:
test.gi_frame

<frame at 0x7fffb9e40ed0, file '/tmp/ipykernel_526/839852198.py', line 6, code my_generator>

In [208]:
a = range(4)

In [214]:
for i in range(1, 5):
    print(i)

1
2
3
4


In [251]:
import sys


def trace_calls(frame, event, arg):
    if event == "line":
        if frame.f_code.co_name == "my_generator" and frame.f_lineno == 3:
            # Force the execution to "jump" back to line 1
            frame.f_lineno = 1
    return trace_calls


def my_generator():
    a = 1
    print("hola")
    yield a
    b = a + 1
    for i in range(3, 5):
        yield i


# Set the trace function
sys.settrace(trace_calls)

test = my_generator()
a = next(test)  # Will execute the first yield (line 3)
print(a)
a = next(test)  # Will execute the second yield (line 5)
print(a)
# Turn off tracing after we're done
sys.settrace(a.gi_frame)

hola
1
3


AttributeError: 'int' object has no attribute 'gi_frame'

In [250]:
next(test)

4

In [307]:
def avg():
    i = 1
    new = yield None
    while True:
        new += yield new / i
        i += 1


t = avg()
next(t)
t.send(0)  # 0/1 =0

t.send(100)  # (0+100)/2 = 50

# provisional value
t.send(100)  # (0+100+ 100)/3 = 66.66666666666667

# definitive
t.send(80)  # (0+100+ 80)/3 = 60

70.0

60.0

In [309]:
def avg():
    i = 1
    new = yield None
    while True:
        new += yield new / i
        i += 1


t = avg()
next(t)
t.send(0)  # 0/1 =0

t.send(100)  # (0+100)/2 = 50

# provisional value
t.send(80)  # (0+100+ 100)/3 = 66.66666666666667

# definitive
t.send(80)  # (0+100+ 80)/3 = 60

65.0

In [338]:
frame = t.gi_frame
print(frame.f_locals)

{'i': 10, 'new': 740}


In [339]:
frame.f_locals

{'i': 10, 'new': 740}

In [328]:
for k, v in {"i": 4, "new": 260}.items():
    frame.f_locals[k] = v
print(frame.f_locals)

{'i': 10, 'new': 740}


In [330]:
frame.f_locals["i"] = 2

In [325]:
t.send(80)

74.0

In [377]:
import sys
from types import FrameType


def tracer(frame: FrameType, event: str, arg):
    if event == "line":
        # Print the line being executed
        code = linecache.getline(frame.f_code.co_filename, frame.f_lineno).strip()
        # print(f"Executing line {frame.f_lineno}: {linecache.getline(frame.f_code.co_filename, frame.f_lineno).strip()}") #   - {frame.f_code.co_name}
        # code frame.f_code.co_name
        # Check if we're at the print line and loop isn't finished
        if code == "test()":
            print(frame.f_lineno)
            if tracer.counter < 5:  # Control the number of iterations
                tracer.counter += 1
                frame.f_lineno = 4  # Jump back to "a += 1"
    return tracer


def test():
    a = 1
    a += 1  # Increment
    print(a)  # Print value
    # Execution should go back to a+=1 five times


import linecache

tracer.counter = 0

# Apply the tracer
sys.settrace(tracer)
try:
    test()
finally:
    sys.settrace(None)

30
30
30
30
30
30
2


In [390]:
def avg():
    i = 1
    new = yield None
    while True:
        new += yield new / i
        i += 1


t = avg()
next(t)
t.send(0)  # 0/1 =0

t.send(100)  # (0+100)/2 = 50

50.0

In [391]:
source_code = inspect.getsource(t.gi_code)
print(source_code)

def avg():
    i = 1
    new = yield None
    while True:
        new+= yield new/i
        i+=1



In [399]:
line = t.gi_frame.f_lineno - 1

In [414]:
# provisional value
t.send(80)  # (0+100+ 100)/3 = 66.66666666666667

# definitive
t.send(80)  # (0+100+ 80)/3 = 60

65.0

In [1]:
from helpers import (
    Code,
    get_implicit_iterator_in_for_loop_everywhere,
    get_implicit_iterator_in_for_loop,
    tests,
)
import inspect

testing


In [2]:
def test(a):
    a = a + 2
    yield a
    while True:
        a += 1
        if a > 5:
            yield a
        a += 2

In [19]:
def my_copy(generator):
    code = Code.from_generator(generator)
    args = generator.gi_frame.f_locals.copy()  # need to pass all globals that may be used as well.
    args = ",".join(f"{k}={v}" for k, v in args.items())
    return _my_copy(code, args)


def _my_copy(code, args, current=None):
    # function def, not really needed, but fine

    new_code = current if current else Code(f"def saved({args}):\n")
    line = code.running_line + 1

    # start with the most extenral block
    if code.block in ["if", "else:", "elif"]:
        # if if is inside a while
        new_code += "    if True:"
        new_code += "        pass"
    elif code.block == "while":
        # If while is found the rest of the iteration goes to the top. as is the starting point
        new_code += code.scope_after.dedent.indent
        line = code.block_start_line
    elif code.block == "for":
        # worst case as we need to capture the iterator with magic
        new_code += code.scope_after.dedent.indent
        new_code += "for NOTIMPLEMENTED"
        line = code.block_start_line + 1

    # main code
    new_code += code[line:]

    # executor
    new_code += f"n = saved({args})"
    print(new_code.code)
    exec(new_code.code)
    return eval("n")

In [20]:
t1 = test(10)
next(t1)
next(t1)

13

In [21]:
code = Code.from_generator(t1)

In [22]:
code

def test(a):
    a = a +2
    yield a 
    while True:
        a +=1 
        if a >5:
            yield a 
        a+=2

In [24]:
code.running_line_txt

'            yield a '

In [ ]:
10000 -> 1000 -> 100 
          900 -> 100

----------------------------

if a:
    code1()
    yield b # x
    code2()
elif:
    yield c
else:
    yield d
   
Sol:
yield b # x
code2()

----------------------------
    
if a:
    yield b
elif:
    yield c  # x
else:
    yield d

Sol: 
yield c # x

----------------------------

while True:
    code_a()
    yield a  # x
    code_b()
    
Sol 1:
yield a
code_b()
while True:
    code_a()
    yield a
    code_b()
    
----------------------------

while condition:
    condition = code_a()
    yield a  # x 
    condition = code_b()
    
    
yield a
condition = code_b()
while condition:
    condition = code_a()
    yield a
    condition = code_b()
   
----------------------------
   
b = list...
for a in b:
    code_a() 
    yield a  # x
    code_b()
    

yield a  # x
code_b()
c = b[curr:] # hard to get but can be done in some cases.
for a in c:
    code_a() 
    yield a  # x
    code_b()
    
----------------------------
for a in [1,3,4]: # same as before but I wasn't able to find the iterable of this in the gc yet.
    code_a() 
    yield a  # x
    code_b()
    
----------------------------

while cond1:
    code1()
    while cond2:
        code2()
        yield b
        code3()
    code4
        

yield b
code3()
while cond2:
    code2()
    yield b
    code3()
code4

while cond1:
    code1()
    while cond2:
        code2()
        yield b
        code3()
    code4
    
    
In general if we have a loop we need to:

1 finish iteration
2 finish loop from the iteration we are on
3.1 if outter loop, finish outer iteration
    3.2 finish outer loop from the iteration we are on and all iterations in the inner one
4.1 if outter outter loop, finish outter outter iteration
    4.2 finish outer outer loop from the iteration we are on and all iterations in both inner ones
repeat 4 four outterx3 etc.

In [ ]:
T